In [118]:
from riotwatcher import LolWatcher, ApiError
from riot_api import key
import pandas as pd
from datetime import datetime

region = 'euw1'

lol = LolWatcher(key)

In [2]:
me = lol.summoner.by_name(region,'Katiounette')
puuid = me['puuid']

In [3]:
matchs = lol.match.matchlist_by_puuid(region,puuid, count = 100)
i = 100
while (lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i) != []):
    matchs.extend(lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i))
    i = i + 100

In [164]:
matchData = pd.DataFrame({'playerID' : [],'matchID' : [], 'duration (s)' : [], 'start' : [], 'end' : [], 'championName' : [], 'championId' : [], 'pings' : [], 'ePings' : []})
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings


In [165]:
for j,i in enumerate(matchs):
    p = lol.match.by_id(region,i)
    for player in p['info']['participants']:
        if player['puuid'] == puuid:
            cName = player['championName']
            cId = player['championId']
            ePing = player['enemyMissingPings']
            pings = player["allInPings"] + player["assistMePings"] + player["baitPings"] + player["basicPings"] + player["commandPings"] + player["dangerPings"] + player["getBackPings"] + player["holdPings"] + player["needVisionPings"] + player["onMyWayPings"] + player["pushPings"] + player['visionClearedPings'] + player['enemyMissingPings'] + player['enemyVisionPings']
    print(pd.DataFrame([[puuid,p['metadata']['matchId'],p['info']['gameDuration'], p['info']['gameStartTimestamp'], p['info']['gameEndTimestamp'], cName, cId, pings, ePing]]))
    matchData.loc[j,:] = [puuid,p['metadata']['matchId'],p['info']['gameDuration'], p['info']['gameStartTimestamp'], p['info']['gameEndTimestamp'], cName, cId, pings, ePing]
    break

                                                   0                1     2  \
0  MT9onfZzAdUlo_etm8mfUioks5OKN07dFQ_z-Y1ojlygmZ...  EUW1_6346539622  1711   

               3              4            5   6  7  8  
0  1680633816166  1680635527190  MissFortune  21  5  2  


In [166]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings
0,MT9onfZzAdUlo_etm8mfUioks5OKN07dFQ_z-Y1ojlygmZ...,EUW1_6346539622,1711.0,1.680634e+12,1.680636e+12,MissFortune,21.0,5.0,2.0


In [167]:
for i in range(matchData.shape[0]):
    matchData.loc[i,'start'] = pd.Timestamp(datetime.fromtimestamp(matchData.loc[i,'start']/1000))
    matchData.loc[i,'end'] = pd.Timestamp(datetime.fromtimestamp(matchData.loc[i,'end']/1000))
    matchData.loc[i,'weekDay'] = matchData.loc[i,'start'].day_name()
    matchData.loc[i,'startHour'] = matchData.loc[i,'start'].hour

In [168]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings,weekDay,startHour
0,MT9onfZzAdUlo_etm8mfUioks5OKN07dFQ_z-Y1ojlygmZ...,EUW1_6346539622,1711.0,2023-04-04 20:43:36.166000,2023-04-04 21:12:07.190000,MissFortune,21.0,5.0,2.0,Tuesday,20.0
